In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np 

In [2]:
df=pd.read_csv('diabetes.csv')
df=df.drop(columns='family_histroy')
df.replace(4.860753, np.nan, inplace=True)
df.dropna(inplace=True)
df["smoking"] = df["smoking"].replace({1.0: 0, 2.0: 0, 3.0: 1})
df_copy=df
df_copy["smoking"] = df["smoking"].astype(int)


X = df_copy.drop('smoking', axis=1)
y = df_copy['smoking']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [ ]:

class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

model = MLP(input_dim=X_train.shape[1])


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


model.eval()
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_labels = (y_pred > 0.5).float()
    acc = accuracy_score(y_test, y_pred_labels)
    print(f"\nTest Accuracy: {acc:.4f}")

Epoch 0, Loss: 0.7588
Epoch 10, Loss: 0.6922
Epoch 20, Loss: 0.6386
Epoch 30, Loss: 0.5834
Epoch 40, Loss: 0.5320

Test Accuracy: 0.7101
